In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import pandas as pd
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"


X_train = pd.read_csv("../datasets/X_train(15052022)-(1.00).csv")
X_test = pd.read_csv("../datasets/X_test(15052022)-(1.00).csv")
y_train = pd.read_csv("../datasets/y_train(15052022)-(1.00).csv")
y_test = pd.read_csv("../datasets/y_test(15052022)-(1.00).csv")


In [2]:
def build_model(hp):
    """
    Builds model and sets up hyperparameter space to search.
    
    Parameters
    ----------
    hp : HyperParameter object
        Configures hyperparameters to tune.
        
    Returns
    -------
    model : keras model
        Compiled model with hyperparameters to tune.
    """
    # Initialize sequential API and start building model.
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(195, )))
    
    # Tune the number of hidden layers and units in each.
    # Number of hidden layers: 1 - 5
    # Number of Units: 32 - 512 with stepsize of 32
    for i in range(1, hp.Int("num_layers", 2, 6)):
        model.add(
            keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
                activation="relu")
            )
        
        # Tune dropout layer with values from 0 - 0.3 with stepsize of 0.1.
        model.add(keras.layers.Dropout(hp.Float("dropout_" + str(i), 0, 0.3, step=0.1)))
    
    # Add output layer.
    model.add(keras.layers.Dense(units=10, activation="softmax"))
    
    # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    
    # Define optimizer, loss, and metrics
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=["accuracy"])
    
    return model

In [3]:
# Build baseline model with Sequential API
b_model = keras.Sequential()
b_model.add(keras.layers.Flatten(input_shape=(195, )))
b_model.add(keras.layers.Dense(units=512, activation='relu', name='dense_1'))
b_model.add(keras.layers.Dropout(0.2))
b_model.add(keras.layers.Dense(10, activation='softmax'))

In [4]:
# Set training parameters
b_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [5]:
# Number of epochs
NUM_EPOCHS = 20

# Early stopping set after 5 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Train model
b_model.fit(X_train, y_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early], verbose=2)


Epoch 1/20
608/608 - 4s - loss: 0.2597 - accuracy: 0.9134 - val_loss: 0.2333 - val_accuracy: 0.9175 - 4s/epoch - 6ms/step
Epoch 2/20
608/608 - 1s - loss: 0.2339 - accuracy: 0.9175 - val_loss: 0.2302 - val_accuracy: 0.9181 - 1s/epoch - 2ms/step
Epoch 3/20
608/608 - 1s - loss: 0.2271 - accuracy: 0.9177 - val_loss: 0.2325 - val_accuracy: 0.9158 - 1s/epoch - 2ms/step
Epoch 4/20
608/608 - 1s - loss: 0.2219 - accuracy: 0.9213 - val_loss: 0.2318 - val_accuracy: 0.9187 - 1s/epoch - 2ms/step
Epoch 5/20
608/608 - 1s - loss: 0.2165 - accuracy: 0.9214 - val_loss: 0.2362 - val_accuracy: 0.9162 - 1s/epoch - 2ms/step
Epoch 6/20
608/608 - 1s - loss: 0.2080 - accuracy: 0.9249 - val_loss: 0.2366 - val_accuracy: 0.9183 - 1s/epoch - 2ms/step
Epoch 7/20
608/608 - 1s - loss: 0.2021 - accuracy: 0.9272 - val_loss: 0.2401 - val_accuracy: 0.9160 - 1s/epoch - 2ms/step


In [7]:
import pandas as pd

def evaluate_model(model, X_test, y_test):
    """
    evaluate model on test set and show results in dataframe.
    
    Parameters
    ----------
    model : keras model
        trained keras model.
    X_test : numpy array
        Features of holdout set.
    y_test : numpy array
        Labels of holdout set.
        
    Returns
    -------
    display_df : DataFrame
        Pandas dataframe containing evaluation results.
    """
    eval_dict = model.evaluate(X_test, y_test, return_dict=True)
    
    display_df = pd.DataFrame([eval_dict.values()], columns=[list(eval_dict.keys())])
    
    return display_df

# Evaluate model on test set and add results to dataframe
results = evaluate_model(b_model, X_test, y_test)

# Set index to 'Baseline'
results.index = ['Baseline']

# Display results
results.head()

374/374 [==============================] - 1s 2ms/step - loss: 0.2439 - accuracy: 0.9149


,loss,accuracy
Baseline,0.243883,0.914926


In [8]:
# Instantiate the tuner
tuner = kt.Hyperband(build_model,
                     objective="val_accuracy",
                     max_epochs=50,
                     factor=3,
                     hyperband_iterations=10,
                     directory="kt_dir",
                     project_name="kt_hyperband",)

In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 12
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
dropout_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
dropout_2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': None}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
dropout_3 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': None}
units_4 (Int)
{'defau

In [10]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [11]:
tuner.search(X_train, y_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early], verbose=2)

# # Get the optimal hyperparameters
# best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

# print(f"""
# The hyperparameter search is complete. The optimal number of units in the first densely-connected
# layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
# is {best_hps.get('learning_rate')}.
# """)

Trial 307 Complete [00h 00m 17s]
val_accuracy: 0.9174727201461792

Best val_accuracy So Far: 0.9211772084236145
Total elapsed time: 00h 01m 35s

Search: Running Trial #308

Value             |Best Value So Far |Hyperparameter
5                 |4                 |num_layers
320               |384               |units_1
0.2               |0.2               |dropout_1
0.001             |0.0001            |learning_rate
224               |480               |units_2
0.2               |0.2               |dropout_2
224               |352               |units_3
0.1               |0.1               |dropout_3
416               |384               |units_4
0                 |0.2               |dropout_4
128               |448               |units_5
0.1               |0.2               |dropout_5
20                |7                 |tuner/epochs
7                 |0                 |tuner/initial_epoch
1                 |1                 |tuner/bracket
1                 |0                 |tune

KeyboardInterrupt: 

In [11]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [12]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
h_model = tuner.hypermodel.build(best_hps)
history = h_model.fit(X_train, y_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early], verbose=2)


val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/20
608/608 - 2s - loss: 0.3211 - accuracy: 0.9067 - val_loss: 0.2346 - val_accuracy: 0.9181 - 2s/epoch - 4ms/step
Epoch 2/20
608/608 - 2s - loss: 0.2391 - accuracy: 0.9143 - val_loss: 0.2331 - val_accuracy: 0.9158 - 2s/epoch - 3ms/step
Epoch 3/20
608/608 - 2s - loss: 0.2343 - accuracy: 0.9160 - val_loss: 0.2316 - val_accuracy: 0.9144 - 2s/epoch - 3ms/step
Epoch 4/20
608/608 - 2s - loss: 0.2292 - accuracy: 0.9180 - val_loss: 0.2338 - val_accuracy: 0.9164 - 2s/epoch - 3ms/step
Epoch 5/20
608/608 - 2s - loss: 0.2264 - accuracy: 0.9193 - val_loss: 0.2292 - val_accuracy: 0.9173 - 2s/epoch - 3ms/step
Epoch 6/20
608/608 - 2s - loss: 0.2224 - accuracy: 0.9200 - val_loss: 0.2304 - val_accuracy: 0.9181 - 2s/epoch - 3ms/step
Epoch 7/20
608/608 - 2s - loss: 0.2192 - accuracy: 0.9211 - val_loss: 0.2318 - val_accuracy: 0.9181 - 2s/epoch - 3ms/step
Epoch 8/20
608/608 - 2s - loss: 0.2177 - accuracy: 0.9207 - val_loss: 0.2321 - val_accuracy: 0.9166 - 2s/epoch - 3ms/step
Epoch 9/20
608/608 - 2s 

In [13]:
# Evaluate model on test set
hyper_df = evaluate_model(h_model, X_test, y_test)

# Set index to hypertuned
hyper_df.index = ["Hypertuned"]

# Append results in dataframe
results.append(hyper_df)

374/374 [==============================] - 1s 2ms/step - loss: 0.2391 - accuracy: 0.9163


C:\Users\Robert\AppData\Local\Temp\ipykernel_1160\1061095111.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results.append(hyper_df)


,loss,accuracy
Baseline,0.247332,0.912251
Hypertuned,0.239135,0.916263


In [16]:
# Define hypermodel with input shape and number of classes
hypermodel = kt.applications.HyperResNet(input_shape=(195, ), classes=10)

# Instantiate tuner with bayesian optimzation search algorithm and our hypermodel
tuner = kt.tuners.BayesianOptimization(
    hypermodel,
    objective='val_accuracy',
    max_trials=3,
    directory="kt_dir",
    project_name="kt_bayes_resnet")

ValueError: Input 0 of layer "conv1_pad" is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: (None, 195)

In [ ]:
# ResNet expects shape of convolutional layer
X_train_res = X_train.reshape(len(X_train), 28, 28, 1)
X_test_res = X_test.reshape(len(X_test), 28, 28, 1)

# ResNet expects one-hot encoded labels
y_train_res = keras.utils.to_categorical(y_train)
y_test_res = keras.utils.to_categorical(y_test)

tuner.search(X_train_res, y_train_res, epochs=NUM_EPOCHS, validation_split=0.2, verbose=2)


In [19]:
# Get the optimal hyperparameters from the results
best_hps=tuner.get_best_hyperparameters()[0]

resnet_model = tuner.hypermodel.build(best_hps)

# Train the hypertuned model
resnet_model.fit(X_train_res, y_train_res, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early], verbose=2)


NameError: name 'X_train_res' is not defined

In [ ]:
resnet_df = evaluate_model(resnet_model, X_test_res, y_test_res)

resnet_df.index = ["HyperResNet"]

results.append(resnet_df)